In [6]:
# import numpy as np

# # 1. DEFINE THE WORLD
# grid_size = 5
# start_pos = (0, 0)  # Top-left corner (row, column)
# goal_pos = (4, 4)   # Bottom-right corner

# # 2. DEFINE ACTIONS: Up, Down, Left, Right
# actions = {
#     0: (-1, 0),  # Up: subtract 1 from row
#     1: (1, 0),   # Down: add 1 to row
#     2: (0, -1),  # Left: subtract 1 from column
#     3: (0, 1)    # Right: add 1 to column
# }
# num_actions = len(actions)

# # 3. THE AGENT'S BRAIN: Q-Table
# # The table has one row for every possible cell (5x5 = 25 states)
# # and one column for each action (4 actions).
# q_table = np.zeros((grid_size * grid_size, num_actions))
# print("Agent's starting Q-Table (empty brain):")
# print(q_table)

# # 4. A FUNCTION TO SIMULATE A STEP
# def take_action(state, action_id):
#     """Simulates moving the agent in the grid.
#     Returns: new_state, reward, done"""
#     # Convert state number (0-24) back to (row, col) coordinates
#     row = state // grid_size  # Integer division
#     col = state % grid_size   # Remainder

#     # Get the move (e.g., (0, 1) for Right)
#     move = actions[action_id]
#     new_row = row + move[0]
#     new_col = col + move[1]

#     # Check if the new position is valid (inside the grid)
#     if 0 <= new_row < grid_size and 0 <= new_col < grid_size:
#         new_state = new_row * grid_size + new_col # Convert back to a state number
#     else:
#         new_state = state  # If move is invalid, stay in place

#     # Check if the new state is the GOAL
#     if new_state == (goal_pos[0] * grid_size + goal_pos[1]):
#         reward = 10
#         done = True
#         print(f"🎉 Goal reached! Agent gets a big reward: {reward}")
#     else:
#         reward = -1  # Small punishment for every step to encourage faster paths
#         done = False

#     return new_state, reward, done

# # 5. LET'S MANUALLY TEST ONE STEP
# print("\n--- Let's test one step ---")
# current_state = 0 # State 0 is (0,0) - the start
# print(f"Agent is at state: {current_state} (which is position {start_pos})")
# chosen_action = 3 # Let's choose action 3: "Right"
# print(f"Agent chooses action: {chosen_action} (Right)")

# new_state, reward, done = take_action(current_state, chosen_action)
# print(f"Result: New State: {new_state}, Reward: {reward}, Episode Done? {done}")

In [10]:
import numpy as np
from tqdm import tqdm # For the progress bar

# 1. DEFINE THE WORLD
grid_size = 5
start_state = 0  # Top-left corner (0, 0)
goal_state = 20  # Bottom-right corner (4, 4)

# 2. DEFINE ACTIONS: Up, Down, Left, Right
actions = {
    0: (-1, 0),  # Up: subtract 1 from row
    1: (1, 0),   # Down: add 1 to row
    2: (0, -1),  # Left: subtract 1 from column
    3: (0, 1)    # Right: add 1 to column
}
num_actions = len(actions)

# 3. THE AGENT'S BRAIN: Q-Table
# The table has one row for every possible cell (5x5 = 25 states)
# and one column for each action (4 actions).
q_table = np.zeros((grid_size * grid_size, num_actions))

# 4. A FUNCTION TO SIMULATE A STEP
def take_action(state, action_id):
    """Simulates moving the agent in the grid.
    Returns: new_state, reward, done"""
    # Convert state number (0-24) back to (row, col) coordinates
    row = state // grid_size
    col = state % grid_size

    # Get the move (e.g., (0, 1) for Right)
    move = actions[action_id]
    new_row = row + move[0]
    new_col = col + move[1]

    # Check if the new position is valid (inside the grid)
    if 0 <= new_row < grid_size and 0 <= new_col < grid_size:
        new_state = new_row * grid_size + new_col
    else:
        new_state = state  # If move is invalid, stay in place

    # Check if the new state is the GOAL
    if new_state == goal_state:
        reward = 100  # Big reward for reaching the goal!
        done = True
    else:
        reward = -1   # Small punishment for every step to encourage faster paths
        done = False

    return new_state, reward, done

# 5. HYPERPARAMETERS FOR THE LEARNING ALGORITHM
learning_rate = 0.1
discount_factor = 0.99  # How much we care about future rewards
exploration_rate = 0.1  # Epsilon: 10% chance to explore
num_episodes = 20000    # How many full games to play

# 6. MAIN TRAINING LOOP
print("🟢 Starting Training...")
# Lists to track performance
rewards_per_episode = []
steps_per_episode = []

for episode in tqdm(range(num_episodes), desc="Training", unit="episode"):
    # Reset the game for a new episode
    current_state = start_state
    done = False
    total_reward = 0
    steps = 0

    # Play one full game (episode) until done
    while not done:
        steps += 1
        # Choose action: Explore or Exploit?
        if np.random.random() < exploration_rate:
            action = np.random.randint(num_actions)  # Explore: random action
        else:
            action = np.argmax(q_table[current_state]) # Exploit: best action

        # Take the action, get new state and reward
        new_state, reward, done = take_action(current_state, action)
        total_reward += reward

        # Update Q-table using the Q-learning formula
        old_value = q_table[current_state, action]
        next_max = np.max(q_table[new_state])
        
        # Q-learning formula: New Q = Old Q + LR * (Reward + DF * Best Future Q - Old Q)
        new_value = old_value + learning_rate * (reward + discount_factor * next_max - old_value)
        q_table[current_state, action] = new_value

        # Move to the new state
        current_state = new_state

    # Record the results of this episode
    rewards_per_episode.append(total_reward)
    steps_per_episode.append(steps)

# 7. SHOW FINAL RESULTS
print("\n✅ Training Finished!")
print(f"Average steps per episode (last 1000): {np.mean(steps_per_episode[-1000:]):.2f}")
print(f"Average reward per episode (last 1000): {np.mean(rewards_per_episode[-1000:]):.2f}")

# 8. SHOW THE LEARNED POLICY (what the agent will do in each state)
print("\n🧠 Agent's Learned Policy (best action for each state):")
for row in range(grid_size):
    for col in range(grid_size):
        state = row * grid_size + col
        best_action = np.argmax(q_table[state])
        action_symbol = ['↑', '↓', '←', '→'][best_action]
        print(f"{action_symbol} ", end='')
    print()  # New line after each row

# 9. (Optional) You can add plotting here to see the improvement over time!

🟢 Starting Training...


Training: 100%|██████████| 20000/20000 [00:00<00:00, 46793.57episode/s]


✅ Training Finished!
Average steps per episode (last 1000): 4.57
Average reward per episode (last 1000): 96.43

🧠 Agent's Learned Policy (best action for each state):
↓ ← ← ← ↓ 
↓ ↑ ← ← ← 
↓ ↓ ← ← ↑ 
↓ ← ← ↑ → 
↑ ← ↑ ← ← 


In [ ]:
# 10. VISUALIZATION: WATCH THE TRAINED AGENT PLAY
def draw_grid(state, path=[]):
    """Draws the grid world, showing the agent's current position and path."""
    print("\n" + "-" * (grid_size * 2 + 3))
    for row in range(grid_size):
        print("|", end="")
        for col in range(grid_size):
            current_pos = row * grid_size + col
            if current_pos == state:
                print("🤖", end="")  # Agent
            elif current_pos == start_state:
                print("🚦", end="")  # Start
            elif current_pos == goal_state:
                print("🎯", end="")  # Goal
            elif current_pos in path:
                print("◼️", end="")  # Visited path
            else:
                print("◻️", end="")  # Empty cell
        print("|")
    print("-" * (grid_size * 2 + 3))

def run_episode_with_visualization():
    """Runs one episode showing each step visually."""
    print("\n🎬 Now watching trained agent play...")
    current_state = start_state
    done = False
    path = [current_state]
    
    while not done:
        # Clear screen for better animation (works in most terminals)
        print("\033[H\033[J", end="")
        
        print(f"Step {len(path)}:")
        draw_grid(current_state, path)
        
        # Choose best action from Q-table
        action = np.argmax(q_table[current_state])
        action_symbol = ['↑', '↓', '←', '→'][action]
        print(f"Action: {action_symbol}")
        
        # Take the action
        new_state, reward, done = take_action(current_state, action)
        
        # Add to path and update state
        if new_state not in path:  # Avoid duplicates in path
            path.append(new_state)
        current_state = new_state
        
        # Wait a bit so human can see
        input("Press Enter for next step...")
    
    # Show final result
    print("\033[H\033[J", end="")
    print("✅ Episode Complete!")
    draw_grid(current_state, path)
    print(f"T tal steps: {len(path)}")
    print(f"Final reward: {reward}")

# Run the visualization
run_episode_with_visualization()


🎬 Now watching trained agent play...
Step 1:

-------------
|🤖◻️◻️◻️◻️|
|◻️◻️◻️◻️◻️|
|◻️◻️◻️◻️◻️|
|◻️◻️◻️◻️◻️|
|🎯◻️◻️◻️◻️|
-------------
Action: ↓
Step 2:

-------------
|🚦◻️◻️◻️◻️|
|🤖◻️◻️◻️◻️|
|◻️◻️◻️◻️◻️|
|◻️◻️◻️◻️◻️|
|🎯◻️◻️◻️◻️|
-------------
Action: ↓
Step 3:

-------------
|🚦◻️◻️◻️◻️|
|◼️◻️◻️◻️◻️|
|🤖◻️◻️◻️◻️|
|◻️◻️◻️◻️◻️|
|🎯◻️◻️◻️◻️|
-------------
Action: ↓
Step 4:

-------------
|🚦◻️◻️◻️◻️|
|◼️◻️◻️◻️◻️|
|◼️◻️◻️◻️◻️|
|🤖◻️◻️◻️◻️|
|🎯◻️◻️◻️◻️|
-------------
Action: ↓
✅ Episode Complete!

-------------
|🚦◻️◻️◻️◻️|
|◼️◻️◻️◻️◻️|
|◼️◻️◻️◻️◻️|
|◼️◻️◻️◻️◻️|
|🤖◻️◻️◻️◻️|
-------------
Total steps: 5
Final reward: 100
